In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from nltk.corpus import stopwords
import nltk

nltk.download('wordnet')




[nltk_data] Downloading package wordnet to /home/xavient/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#Load the data using pandas : Create a DataFrame named df, that contains the training data 
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [4]:
df = df.drop(['headline', 'authors', 'link', 'date'], axis=1)


df  = df.sample(frac=0.02)


In [5]:
df.shape

(4017, 2)

In [6]:
df = df.drop_duplicates(subset=['short_description'])

In [7]:
df.shape

(3641, 2)

In [8]:
df.head()

,category,short_description
182967,MONEY,"On Tuesday, Ortega's net worth rose to $47.4 b..."
140752,TRAVEL,Custer State Park is located just an hour's dr...
106148,HEALTHY LIVING,Your chubby teen self whom you felt ashamed of...
128723,FOOD & DRINK,Some processed foods are most enjoyable when c...
158571,FOOD & DRINK,It's time to put it in the spotlight.


In [9]:
#df_init.shape
len(df.groupby('category').count())

41

In [10]:
df['Category'] = df['category']
df['Text'] = df['short_description']

df = df.drop(['category', 'short_description'], axis=1)

In [11]:
df.shape

(3641, 2)

In [12]:
 df.dropna(inplace=True)

In [13]:
df.shape

(3641, 2)

In [14]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(text):
    return ' '.join([str(lemmatizer.lemmatize(w)).lower() for w in w_tokenizer.tokenize(text)])

In [15]:
stop = stopwords.words('english')

df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Text'] = df['Text'].apply(lambda x: lemmatize_text(x))


In [16]:
df.head()

,Category,Text
182967,MONEY,"on tuesday, ortega's net worth rose $47.4 bill..."
140752,TRAVEL,custer state park located hour's drive south m...
106148,HEALTHY LIVING,your chubby teen self felt ashamed of? she als...
128723,FOOD & DRINK,some processed food enjoyable consumed veil ig...
158571,FOOD & DRINK,it's time put spotlight.


In [17]:
df['category_id'] = df['Category'].factorize()[0]

#View first 10 entries of category_id, as a sanity check
df['category_id'][0:10]
df.head(10)


,Category,Text,category_id
182967,MONEY,"on tuesday, ortega's net worth rose $47.4 bill...",0
140752,TRAVEL,custer state park located hour's drive south m...,1
106148,HEALTHY LIVING,your chubby teen self felt ashamed of? she als...,2
128723,FOOD & DRINK,some processed food enjoyable consumed veil ig...,3
158571,FOOD & DRINK,it's time put spotlight.,3
4284,WORLD NEWS,the activist's son said mother barred getting ...,4
93838,WEIRD NEWS,,5
80808,ENTERTAINMENT,somebody paid attention english class!,6
154493,WELLNESS,we could manage weight health least great reli...,7
35060,WOMEN,“in past year i’ve started ask i really want s...,8


In [18]:
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_id_df.head()


,Category,category_id
182967,MONEY,0
140752,TRAVEL,1
106148,HEALTHY LIVING,2
128723,FOOD & DRINK,3
4284,WORLD NEWS,4


In [19]:
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)


In [20]:
 df.groupby('Category').count()

,Text,category_id
Category,,
ARTS,18,18
ARTS & CULTURE,24,24
BLACK VOICES,79,79
BUSINESS,94,94
COLLEGE,25,25
COMEDY,84,84
CRIME,66,66
CULTURE & ARTS,23,23
DIVORCE,59,59


In [21]:
higher = df.groupby('Category').count()['Text'].max(axis=0)
higher

564

In [22]:
from sklearn.utils import resample

for category in category_id_df['Category']:
    
    min_imblanced = df[df.Category==category]
    other_imblanced = df[df.Category!=category]
    
    up_scale = resample(min_imblanced, replace=True, n_samples=higher, random_state=123)
    df = pd.concat([other_imblanced, up_scale])
    #print(other_imblanced)
    
    

In [23]:
df.head()

,Category,Text,category_id
170753,MONEY,"taking money time tenet faith. i believe god, ...",0
134475,MONEY,by flipping script place scarcity hopelessness...,0
134828,MONEY,you may find easier control budget cash debit ...,0
134475,MONEY,by flipping script place scarcity hopelessness...,0
191295,MONEY,transferring home child clumsy method trying a...,0


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

# tfidf = HashingVectorizer( n_features=2**10,  encoding='latin-1', ngram_range=(1, 2), stop_words='english')
# tfidf = CountVectorizer( min_df=5,  encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

index = 0
#while index < len(df.Text):
features = tfidf.fit_transform(df.Text) # Remaps the words in the 1490 articles in the text column of 
                            # data frame into features (superset of words) with an importance assigned 
                                                  # based on each words frequency in the document and across documents

# labels = df.category_id                           # represents the category of each of the 1490 articles

# features.shape # How many features are there ? 



In [25]:
features

<23124x23450 sparse matrix of type '<class 'numpy.float64'>'
	with 440662 stored elements in Compressed Sparse Row format>

In [26]:
type(features)

scipy.sparse.csr.csr_matrix

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score


models = [
    RandomForestClassifier(n_estimators=200, max_depth=100, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=30),
    SGDClassifier(loss="modified_huber", penalty="l2", max_iter=5),
    LinearSVC(random_state=0, tol=1e-5),
]


In [28]:
CV = 5  # Cross Validate with 5 different folds of 20% data ( 80-20 split with 5 folds )
#Create a data frame that will store the results for all 5 trials of the 3 different models
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = [] # Initially all entries are empty


In [29]:


# #For each Algorithm 
# for model in models:
#   model_name = model.__class__.__name__
#   # create 5 models with different 20% test sets, and store their accuracies
#   accuracies = cross_val_score(model, features.toarray(), df.category_id, scoring='accuracy', cv=CV)
#   # Append all 5 accuracies into the entries list ( after all 3 models are run, there will be 3x5 = 15 entries)
#   for fold_idx, accuracy in enumerate(accuracies):
#     entries.append((model_name, fold_idx, accuracy))



In [30]:
# cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [31]:
# import seaborn as sns

# sns.boxplot(x='model_name', y='accuracy', data=cv_df)
# sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
#               size=8, jitter=True, edgecolor="gray", linewidth=2)

In [32]:
# cv_df.groupby('model_name').accuracy.mean()

In [33]:
# cv_df

In [34]:
df.category_id

170753     0
134475     0
134828     0
134475     0
191295     0
          ..
100415    40
104870    40
124273    40
118342    40
108252    40
Name: category_id, Length: 23124, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split

model = LogisticRegression(random_state=0) 

#Split Data 
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features.toarray(), df.category_id, df.index, test_size=0.33, random_state=42)

#Train Algorithm
model.fit(X_train, y_train)

# Make Predictions
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)

TypeError: __init__() got an unexpected keyword argument 'workers'

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.Category.values, yticklabels=category_id_df.Category.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')

In [ ]:

df.iloc[1].Text

In [ ]:
texts = [df.iloc[1].Text,
         df.iloc[2].Text,
         df.iloc[3].Text,
         df.iloc[4].Text,
         df.iloc[5].Text]
text_features = tfidf.transform(texts)
predictions = model.predict(text_features)
for text, predicted in zip(texts, predictions):
  print('"{}"'.format(text))
  print("  - Predicted as: '{}'".format(id_to_category[predicted]))
  print("")